# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Evaluating food ratings data of UK Establishments for journal Eat Safe, Love

Source file: `establishments.json`

Database name: `uk_food`

Collection name: `establishments`

Terminal import line: `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json'

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [8]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'class_db', 'config', 'fruits_db', 'local', 'met', 'petsitly', 'travel_db', 'uk_food']


In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [25]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658caefe91779842fbc17855'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [4]:
# Create a dictionary for the new restaurant data
new_document = {
    'AddressLine3': 'Greenwich',
    'BusinessName': 'Penang Flavours',
    'BusinessType': 'Restaurant',
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_document)

In [10]:
# Check that the new restaurant was inserted

# define query
query = {'BusinessName': 'Penang Flavours'}

# perform
results = list(establishments.find(query))
pprint(results)

[{'AddressLine3': 'Greenwich',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant',
  '_id': ObjectId('658cb12fc562a6e766835377')}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessType': 1, 'BusinessTypeID': 1, 'BusinessName': 1}
limit = 5

# Cast the results as a list and save the results to a variable
results = list(establishments.find(query, fields).limit(limit))
pprint(results)

[{'BusinessName': 'Refreshment Kiosk',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658caefe91779842fbc1756e')},
 {'BusinessName': 'The Tea Room',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658caefe91779842fbc17571')},
 {'BusinessName': 'Mrs Knotts Tea Room',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658caefe91779842fbc17573')},
 {'BusinessName': 'Boodles',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658caefe91779842fbc17574')},
 {'BusinessName': 'FirstLight Bar & Café',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658caefe91779842fbc17575')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant document with the correct BusinessTypeID:

# define filter to identify what document to update
filter_criteria = {'BusinessName': 'Penang Flavours'}

# define the update operation
update_operation = {
    '$set': {
        'BusinessType': 'Restaurant/Cafe/Canteen',
        'BusinessTypeID': 1,
        'AddressLine2': 'Lambarde Square',
        'AddressLine3': 'Greenwich',
        'AddressLine4': 'London',
        'BusinessName': 'Penang Flavours',
        'geocode.latitude': '51.485459',
        'geocode.longitude': '0.008496'
    }
}

# perform the update
result = establishments.update_one(filter_criteria, update_operation)

# verify - print the number of matched and modified documents
print(f"Matched {result.matched_count} document(s) and modified {result.modified_count} document(s).")

Matched 1 document(s) and modified 1 document(s).


In [13]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
results = list(establishments.find(query))
pprint(results)

[{'AddressLine3': 'Greenwich',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('658cb12fc562a6e766835377')}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover

994

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"

# define filter criteria
filter_criteria = {'LocalAuthorityName': 'Dover'}

# perform the deletion event
result = establishments.delete_many(filter_criteria)

# verify - print the # of deleted documents
print(f"Deleted {result.deleted_count} document(s).")

Deleted 994 document(s).


In [16]:
# Check if any remaining documents include Dover
dover = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover

0

In [39]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('658caefe91779842fbc17855'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [26]:
# Change the data type from String to Decimal for longitude and latitude

# define filter criteria for update
filter_criteria = {}

# define the update operation - convert lat, long strings to int
update_operation = [
    {
        '$set': {
            'geocode.latitude': {'$toDouble': '$geocode.latitude'},
            'geocode.longitude': {'$toDouble': '$geocode.longitude'}
        }
    }
]

# perform the update
result = establishments.update_many(filter_criteria, update_operation)

# verify - print the # of deleted documents
print(f"Modified {result.modified_count} document(s).")

Modified 38786 document(s).


Use `update_many` to convert `RatingValue` to integer numbers.

In [28]:
# Set non 1-5 Rating Values to Null

non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]

# define filter criteria
filter_criteria = {"RatingValue": {"$in": non_ratings}}

# define the update operation
update_operation = [
    {
        '$set': {
            'RatingValue': None
        }
    }
]

# perform the update
result = establishments.update_many(filter_criteria, update_operation)

# verify - print the # of deleted documents
print(f"Modified {result.modified_count} document(s).")

In [31]:
# Change the data type for RatingValue from String to Integer 

# define filter criteria for update
filter_criteria = {}

# define the update operation - convert lat, long strings to int
update_operation = [
    {
        '$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }
    }
]

# perform the update
result = establishments.update_many(filter_criteria, update_operation)

# verify - print the # of deleted documents
print(f"Modified {result.modified_count} document(s).")

Modified 34695 document(s).


In [46]:
print('Check that the rating values are now numbers:')

result1 = establishments.count_documents({'RatingValue': {'$type': 'string'}})
result2 = establishments.count_documents({'RatingValue': {'$type': 'int'}})


print(result1)
print(result2)

print('----------------------')
print('Check that the coordinate values are now numbers:')

result3 = establishments.count_documents({'geocode.latitude': {'$type': 'string'}})
result4 = establishments.count_documents({'geocode.latitude': {'$type': 'double'}})


print(result3)
print(result4)

Check that the rating values are now numbers:
0
34694
----------------------
Check that the coordinate values are now numbers:
0
38785
